# Nevada Recsys based on Sentiment 

In this notebook, I will be creating various Recommender systems, taking different approaches based on Yelp review star ratings.

In [1]:
# Imports 
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
from IPython.display import Image
seed=42

In [2]:
data = pd.read_csv("/home/schubert/DSI/capstone_project/model_notebooks/sent_rec_data_nv")

In [3]:
data.head()

,business_id,user_id,name,sentiment_score
0,--9e1ONYQuAa-CB_Rrw7Tw,xP1IYu2eGfxMWV9tjrurIw,"""Delmonico Steakhouse""",0.405723
1,--9e1ONYQuAa-CB_Rrw7Tw,oFyOUOeGTRZhFPF9uTqrTQ,"""Delmonico Steakhouse""",0.284375
2,--9e1ONYQuAa-CB_Rrw7Tw,2aeNFntqY2QDZLADNo8iQQ,"""Delmonico Steakhouse""",0.383492
3,--9e1ONYQuAa-CB_Rrw7Tw,gmPP4YFrgYsYQqPYokMgFA,"""Delmonico Steakhouse""",0.310536
4,--9e1ONYQuAa-CB_Rrw7Tw,9bxdPvAhP6cuipD5s2UnCg,"""Delmonico Steakhouse""",0.272811


## Approach

![test](recsys_workflow.png)

In [4]:
users_interactions_count_df = data.groupby(['user_id', 'name']).size().groupby('user_id').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 2].reset_index()[['user_id']]
print('# users with at least 2 interactions: %d' % len(users_with_enough_interactions_df))

# users: 357521
# users with at least 2 interactions: 146664


In [7]:
# Recsys Train-Test-Split
train_data, test_data = train_test_split(data,
                                   #stratify=data['sentiment_score'], # we have imbalanced classes, re: J-shaped distribution
                                   test_size=0.20,
                                   random_state=42)

In [8]:
print('Size of training data: %d' % len(train_data))
print('Size of test data: %d' % len(test_data))

Size of training data: 831640
Size of test data: 207911


### Top N Accuracy

In [19]:
# Indexing by user_id  to speed up the searches during evaluation
data_indexed = data.set_index('user_id')
train_indexed = train_data.set_index('user_id')
test_indexed = test_data.set_index('user_id')

In [9]:
def find_reviwed_restaurants(user_id, data):
    '''
    Fuction that matches user_id to the restaurants they've reviewed
    ----
    Parameters:
    user_id: corresponding unique user id 
    data: pandas dataframe containing business_id, name, user_id, stars
    '''
    reviewed_restaurants = data.loc[user_id]['name']
    return set(reviewed_restaurants if type(reviewed_restaurants) == pd.Series else [reviewed_restaurants])

In [10]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class RecsysEvaluator:


    def businesses_not_reviewed_sample(self, user_id, sample_size, seed=42):
        reviewed_restaurants = find_reviwed_restaurants(user_id, data_indexed)
        all_restaurants = set(data['name'])
        non_reviewed_restaurants = all_restaurants - reviewed_restaurants

        random.seed(seed)
        non_reviewed_restaurants_sample = random.sample(non_reviewed_restaurants, sample_size)
        return set(non_reviewed_restaurants_sample)

    def _check_hit_top_n(self, name, recommended_restaurants, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_restaurants) if c == name)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_reviewer(self, model, user_id):
        # Getting the items in test set
        reviewed_restaurants_testset = test_indexed.loc[user_id]
        if type(reviewed_restaurants_testset['name']) == pd.Series:
            person_reviewed_restaurant_testset = set(reviewed_restaurants_testset['name'])
        else:
            person_reviewed_restaurant_testset = set(reviewed_restaurants_testset['name'])  
        reviewed_restaurant_count_testset = len(reviewed_restaurants_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(user_id, 
                                               restaurants_to_ignore = find_reviwed_restaurants(user_id,
                                                                                                data_indexed),
                                               topn=10000000000)

        hits_at_2_count = 0
        hits_at_3_count = 0
        #For each item the user has interacted in test set
        for item_id in person_reviewed_restaurant_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_reviewed_restaurants_sample = self.businesses_not_reviewed_sample(user_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=42)

            #Combining the current interacted item with the 100 random items
            restaurants_to_filter_recs = non_reviewed_restaurants_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['name'].isin(restaurants_to_filter_recs)]                    
            valid_recs = valid_recs_df['name'].values
            
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_2, index_at_2 = self._check_hit_top_n(item_id, valid_recs, 2)
            hits_at_2_count += hit_at_2
            hit_at_3, index_at_3 = self._check_hit_top_n(item_id, valid_recs, 3)
            hits_at_3_count += hit_at_3

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_2 = hits_at_2_count / float(reviewed_restaurant_count_testset)
        recall_at_3 = hits_at_3_count / float(reviewed_restaurant_count_testset)

        person_metrics = {'hits@2_count':[hits_at_2_count], 
                          'hits@3_count':[hits_at_3_count], 
                          'interacted_count': [reviewed_restaurant_count_testset],
                          'recall@2': [recall_at_2],
                          'recall@3': [recall_at_3]}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        reviewers_metrics = []
        for idx, user_id in enumerate(list(test_indexed.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            reviewer_metrics = self.evaluate_model_for_reviewer(model, user_id)  
            reviewer_metrics['_user_id'] = user_id
            reviewers_metrics.append(reviewer_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(reviewer_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_2 = detailed_results_df['hits@2_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_3 = detailed_results_df['hits@3_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@2': global_recall_at_2,
                          'recall@3': global_recall_at_3}    
        return global_metrics, detailed_results_df
    
recsys_evaluator = RecsysEvaluator()

Popularity Model

In [13]:
restaurant_popularity = data.groupby('name')['sentiment_score'].mean().sort_values(ascending=False).reset_index()
restaurant_popularity.head(10)

,name,sentiment_score
0,"""Golden Bar & Restaurant Equipment""",0.669444
1,"""6500 Tavern""",0.631877
2,"""Takeria Puebla""",0.608333
3,"""El Triunfo Restaurant""",0.599421
4,"""T.I.G. BBQ""",0.589494
5,"""Pop N' Sons""",0.577566
6,"""Tacos Los Parados""",0.569158
7,"""Sleepy Wilson's Barbecue""",0.547558
8,"""Husongs Cantina""",0.542396
9,"""Firelight Barn""",0.536445


In [27]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, restaurant_popularity, items_df=None):
        self.restaurant_popularity = restaurant_popularity
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, restaurants_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.restaurant_popularity[~self.restaurant_popularity['name'].isin(restaurants_to_ignore)] \
                               .sort_values('stars', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['eventStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
popularity_model = PopularityRecommender(restaurant_popularity, data)

In [28]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = recsys_evaluator.evaluate_model(popularity_model)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df

Evaluating Popularity recommendation model...
906 users processed

Global metrics:
{'modelName': 'Popularity', 'recall@2': 0.0, 'recall@3': 0.0}


,hits@2_count,hits@3_count,interacted_count,recall@2,recall@3,_user_id
0,0,0,3,0.0,0.0,q4-Nvtr-FlHxAZu66vm-ig


7p4mOmAl0X1P-ACgLcH2Yg

## Trying something different

In [14]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate, KFold, GridSearchCV
from surprise import BaselineOnly, NormalPredictor, accuracy
from surprise import Reader
import os

In [72]:
# path to dataset file
# file_path = os.path.expanduser("/home/schubert/DSI/capstone_project/model_notebooks/star_rec_data_nv.csv")

In [15]:
data.head()

,business_id,user_id,name,sentiment_score
0,--9e1ONYQuAa-CB_Rrw7Tw,xP1IYu2eGfxMWV9tjrurIw,"""Delmonico Steakhouse""",0.405723
1,--9e1ONYQuAa-CB_Rrw7Tw,oFyOUOeGTRZhFPF9uTqrTQ,"""Delmonico Steakhouse""",0.284375
2,--9e1ONYQuAa-CB_Rrw7Tw,2aeNFntqY2QDZLADNo8iQQ,"""Delmonico Steakhouse""",0.383492
3,--9e1ONYQuAa-CB_Rrw7Tw,gmPP4YFrgYsYQqPYokMgFA,"""Delmonico Steakhouse""",0.310536
4,--9e1ONYQuAa-CB_Rrw7Tw,9bxdPvAhP6cuipD5s2UnCg,"""Delmonico Steakhouse""",0.272811


In [80]:
# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
# reader = Reader(line_format='business_id ; name ; user_id ; stars', sep=',')

In [16]:
reader = Reader(rating_scale=(1, 5))

In [17]:
data = Dataset.load_from_df(data[['name', 'user_id', 'sentiment_score']], reader)

In [18]:
cross_validate(BaselineOnly(), data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7839  0.7837  0.7843  0.7848  0.7835  0.7840  0.0005  
MAE (testset)     0.7485  0.7484  0.7488  0.7493  0.7479  0.7486  0.0005  
Fit time          4.49    4.75    4.89    4.81    4.72    4.73    0.13    
Test time         1.82    1.80    1.49    1.81    1.41    1.67    0.18    


{'fit_time': (4.492543458938599,
  4.750595331192017,
  4.889585018157959,
  4.810168504714966,
  4.719566345214844),
 'test_mae': array([0.74853507, 0.74841616, 0.74875461, 0.74931724, 0.74790708]),
 'test_rmse': array([0.78390071, 0.78374458, 0.78429183, 0.78480169, 0.78347207]),
 'test_time': (1.8231899738311768,
  1.7973172664642334,
  1.4912734031677246,
  1.813295841217041,
  1.4105546474456787)}

In [19]:
cross_validate(NormalPredictor(), data, cv=2)

{'fit_time': (0.6187007427215576, 0.8592300415039062),
 'test_mae': array([0.7485425 , 0.74871039]),
 'test_rmse': array([0.78407404, 0.78409467]),
 'test_time': (4.108623266220093, 4.116459369659424)}

In [20]:
# define a cross-validation iterator
kf = KFold(n_splits=3)

algo = SVD()

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.7842
RMSE: 0.7838
RMSE: 0.7841


In [22]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [87]:
# We can now use the algorithm that yields the best rmse:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

In [88]:
results_df = pd.DataFrame.from_dict(gs.cv_results)
results_df.head()

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_lr_all,param_reg_all
0,1.268387,1.265663,1.267009,1.267020,0.001112,7,1.040979,1.037885,1.039616,1.039493,0.001266,6,9.864846,0.056042,2.638301,0.272984,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}",5,0.002,0.4
1,1.274219,1.271427,1.272727,1.272791,0.001141,8,1.048038,1.044812,1.046615,1.046488,0.001320,8,9.491911,0.012535,2.497279,0.302326,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}",5,0.002,0.6
2,1.258668,1.256008,1.257297,1.257324,0.001086,2,1.031735,1.028620,1.030123,1.030159,0.001272,2,9.685466,0.142121,2.497078,0.292577,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}",5,0.005,0.4
3,1.265265,1.262640,1.263859,1.263921,0.001072,5,1.039891,1.036772,1.038325,1.038329,0.001273,5,9.590522,0.135125,2.511860,0.338188,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}",5,0.005,0.6
4,1.260555,1.257970,1.259228,1.259251,0.001055,4,1.033356,1.030274,1.032089,1.031906,0.001265,3,17.901575,0.216289,2.855000,0.072783,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}",10,0.002,0.4


In [54]:
algo = SVD()

In [55]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3344  1.2859  1.3148  1.2859  1.3055  1.3053  0.0184  
MAE (testset)     1.1064  1.0571  1.0919  1.0597  1.0814  1.0793  0.0188  
Fit time          0.24    0.23    0.24    0.24    0.27    0.24    0.02    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


{'fit_time': (0.2445533275604248,
  0.2275230884552002,
  0.23551177978515625,
  0.2358841896057129,
  0.2743501663208008),
 'test_mae': array([1.10641844, 1.05713763, 1.09192113, 1.05970146, 1.0814054 ]),
 'test_rmse': array([1.33441692, 1.28590703, 1.31476337, 1.28588078, 1.30546843]),
 'test_time': (0.0077037811279296875,
  0.006069660186767578,
  0.009871244430541992,
  0.006154060363769531,
  0.00622105598449707)}